In [1]:
%pwd

'c:\\Users\\jsy61\\Documents\\dev\\angledetection\\deep-image-orientation-angle-detection\\notebooks'

In [8]:
# preds2 의 파일 개수 확인

import os
import pandas as pd

path = './preds2'
file_list = os.listdir(path)
print(len(file_list))


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '../preds2'

In [3]:
import os

os.chdir("../")
ROOT_DIR = os.getcwd()

# output_folder 변수를 정의합니다.
folder = os.path.join(ROOT_DIR, "preds2")  # 실제 경로로 변경하세요.

# 찾고자 하는 파일 이름
file_name = "0a5bf6ba56f069c5_processed.jpg"

# 파일 경로 생성
file_path = os.path.join(folder, file_name)

# 파일 존재 여부 확인
if os.path.exists(file_path):
    print(f"파일이 존재합니다: {file_path}")
else:
    print("파일을 찾을 수 없습니다.")

# 다시 하위 폴더로 복귀
os.chdir("./notebooks")


파일이 존재합니다: c:\Users\jsy61\Documents\dev\angledetection\deep-image-orientation-angle-detection\preds2\0a5bf6ba56f069c5_processed.jpg


In [2]:
%pwd

'c:\\Users\\jsy61\\Documents\\dev\\angledetection\\deep-image-orientation-angle-detection\\notebooks'

In [6]:
!pip install konlpy

   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ---------------------------------------- 0.2/19.4 MB 4.1 MB/s eta 0:00:05
   - -------------------------------------- 0.8/19.4 MB 8.5 MB/s eta 0:00:03
   -- ------------------------------------- 1.3/19.4 MB 9.5 MB/s eta 0:00:02
   --- ------------------------------------ 1.9/19.4 MB 10.0 MB/s eta 0:00:02
   ---- ----------------------------------- 2.4/19.4 MB 10.2 MB/s eta 0:00:02
   ------ --------------------------------- 2.9/19.4 MB 10.5 MB/s eta 0:00:02
   ------- -------------------------------- 3.5/19.4 MB 11.2 MB/s eta 0:00:02
   -------- ------------------------------- 4.0/19.4 MB 11.2 MB/s eta 0:00:02
   --------- ------------------------------ 4.6/19.4 MB 11.3 MB/s eta 0:00:02
   ---------- ----------------------------- 5.1/19.4 MB 11.3 MB/s eta 0:00:02
   ----------- ---------------------------- 5.7/19.4 MB 11.3 MB/s eta 0:00:02
   --

In [8]:
import os
import cv2
import numpy as np
from PIL import Image
import csv
from datetime import datetime
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests
import json

# 네이버 클로바 OCR API 설정
CLOVA_OCR_INVOKE_URL = "https://chnnk8i8bo.apigw.ntruss.com/custom/v1/19640/3cf083c28255868b5f464f8ed39b1db95aee54b99b8670a2dbb556a580e9d5da/general"
CLOVA_OCR_SECRET_KEY = "d0RkaUtCZ0Jtd2VNb0x4TVJWT3BiZWVhS0pmRWlnVlc="


# 예상 키워드 및 문서 분류를 위한 키워드 사전
document_keywords = {
    "자동차 등록증": ["자동차등록증", "차대번호", "사용본거지", "자동차", "등록번호","등록증"],
    "진단서": ["진단서", "병명", "발병일", "향후치료소견", "의사", "면허번호"],
    "처방전": ["처방전", "조제시 참고사항", "용법", "조제량", "본인부담금", "보험자부담금"],
    "소견서": ["소견서", "진찰소견", "검사소견", "치료경과", "향후치료계획"],
    "임신 의료비 지급 신청서": ["임신", "출산", "의료비", "지급신청서", "요양기관", "진료과목"],
    "여권": ["REPUBLIC OF KOREA", "PASSPORT", "여권", "국적", "주민등록번호", "만기일"],
    "영수증": ["영수증", "금액", "공급가액", "부가세", "합계금액", "결제방법"],
    "주민등록증": ["주민등록증", "주민등록번호", "발행일", "주소", "시장"],
    "운전면허증": ["운전면허증", "면허번호", "조건", "적성검사", "운전면허", "경찰청"],
    "납부 확인서": ["납부확인서", "납부자", "납부일자", "납부금액", "수납기관"],
    "계좌번호": ["계좌번호", "예금주", "은행명", "계좌", "입금", "은행"],
    "입퇴원 확인서": ["입퇴원확인서", "입원일", "퇴원일", "진단명", "담당의","입원", "퇴원"],
    "외래 진료 확인서": ["외래진료확인서", "진료일시", "진료과목", "진료구분", "담당의사","진료확인서", "질병분류기호", "상병명", "진료기간", "치료내용","통원"],
    "자동차 번호판": ["[0-9]{2,3}[가-힣][0-9]{4}"],
    "자동차 대시보드": ["km/h", "rpm", "fuel", "온도", "ｏｉｌ", "battery"],
    "이력서": ["이력서", "학력사항", "경력사항", "자격증", "생년월일"],
    "진료비 영수증": ["진료비영수증", "환자", "진료과", "수납금액", "진료일"],
    "조제 영수증": ["조제영수증", "약제비", "조제료", "복약지도료", "의약품"]
}

# 형태소 분석기 초기화
okt = Okt()

def get_ocr_result_clova(image):
    """네이버 클로바 OCR API를 사용하여 OCR 결과를 반환합니다."""
    headers = {
        "X-OCR-SECRET": CLOVA_OCR_SECRET_KEY,
        "Content-Type": "application/json"
    }
    
    _, encoded_image = cv2.imencode('.jpg', image)
    image_bytes = encoded_image.tobytes()
    image_base64 = base64.b64encode(image_bytes).decode('utf-8')
    
    data = {
        "images": [
            {
                "format": "jpg",
                "name": "image",
                "data": image_base64
            }
        ],
        "lang": "ko",
        "requestId": "sample_id",
        "timestamp": int(round(time.time() * 1000)),
        "version": "V2"
    }
    
    response = requests.post(CLOVA_OCR_INVOKE_URL, headers=headers, data=json.dumps(data))
    result = response.json()
    
    if "images" in result and len(result["images"]) > 0:
        text = " ".join([field["inferText"] for field in result["images"][0]["fields"]])
        confidence = np.mean([field["inferConfidence"] for field in result["images"][0]["fields"]])
        return text, confidence
    else:
        return "", 0.0

def tokenize_text(text):
    """텍스트를 토큰화합니다."""
    return ' '.join(okt.morphs(text.lower()))

def calculate_similarity(text1, text2):
    """두 텍스트 간의 코사인 유사도를 계산합니다."""
    vectorizer = CountVectorizer().fit_transform([text1, text2])
    return cosine_similarity(vectorizer)[0][1]

def classify_document(text):
    """문서 유형을 추정합니다."""
    tokenized_text = tokenize_text(text)
    max_similarity = 0
    best_match = "Unknown"
    
    for doc_type, keywords in document_keywords.items():
        tokenized_keywords = ' '.join([tokenize_text(keyword) for keyword in keywords])
        similarity = calculate_similarity(tokenized_text, tokenized_keywords)
        if similarity > max_similarity:
            max_similarity = similarity
            best_match = doc_type
    
    return best_match, max_similarity

def process_image(image_path):
    """이미지를 처리하고 최적의 변형을 찾습니다."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"경고: 이미지를 읽을 수 없습니다 - {image_path}")
        return None, None, 0, 0, "", "Unknown"

    transformations = [
        ("원본", image),
        ("90도 회전", cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)),
        ("180도 회전", cv2.rotate(image, cv2.ROTATE_180)),
        ("270도 회전", cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)),
        ("좌우 대칭", cv2.flip(image, 1)),
        ("상하 대칭", cv2.flip(image, 0)),
    ]

    best_score = 0
    best_image = image
    best_transform = "원본"
    best_text = ""
    best_confidence = 0
    best_doc_type = "Unknown"
    best_similarity = 0

    for transform_name, transformed_image in transformations:
        text, confidence = get_ocr_result_clova(transformed_image)
        doc_type, similarity = classify_document(text)
        score = confidence * similarity

        print(f"  {transform_name:20s}: OCR Confidence {confidence:.2f}, 문서 유형 유사도: {similarity:.2f}")
        print(f"    추정 문서 유형: {doc_type}")
        print(f"    텍스트: {text[:100]}..." if len(text) > 100 else f"    텍스트: {text}")

        if score > best_score:
            best_score = score
            best_image = transformed_image
            best_transform = transform_name
            best_text = text
            best_confidence = confidence
            best_doc_type = doc_type
            best_similarity = similarity

    print(f"\n처리 중: {os.path.basename(image_path)}")
    print(f"  최적 변형: {best_transform}")
    print(f"  최종 OCR Confidence: {best_confidence:.2f}")
    print(f"  최종 추정 문서 유형: {best_doc_type} (유사도: {best_similarity:.2f})")
    print(f"  최종 텍스트: {best_text[:100]}..." if len(best_text) > 100 else f"  최종 텍스트: {best_text}")

    return best_image, best_transform, best_confidence, best_similarity, best_text, best_doc_type

def process_folder(input_folder, output_folder, log_file, max_images=100):
    """폴더 내의 이미지를 처리하고 결과를 로깅합니다. 최대 max_images 개수만큼 처리합니다."""
    os.makedirs(output_folder, exist_ok=True)
    
    with open(log_file, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['파일명', '최적 변형', 'OCR Confidence', '문서 유형 유사도', '추정 문서 유형', '텍스트', '처리 시간'])
        
        image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]
        total_images = min(len(image_files), max_images)
        processed_images = 0
        
        for filename in image_files[:max_images]:
            input_path = os.path.join(input_folder, filename)
            print(f"\n처리 시작 ({processed_images + 1}/{total_images}): {filename}")
            
            start_time = time.time()
            result = process_image(input_path)
            end_time = time.time()
            
            if result[0] is None:
                print(f"오류: 이미지 처리 실패 - {filename}")
                continue
            
            best_image, best_transform, confidence, similarity, text, doc_type = result
            
            processing_time = end_time - start_time
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            output_path = os.path.join(output_folder, f"processed_{filename}")
            cv2.imwrite(output_path, best_image)
            
            csv_writer.writerow([filename, best_transform, confidence, similarity, doc_type, text, timestamp])
            
            processed_images += 1
            print(f"처리 완료: {processed_images}/{total_images} ({processed_images/total_images*100:.2f}%)")
            print(f"처리 시간: {processing_time:.2f}초")
            
            # 사용자에게 중단 여부를 묻습니다
            if processed_images % 10 == 0:  # 10개 처리할 때마다 물어봅니다
                user_input = input("계속 처리하시겠습니까? (y/n): ")
                if user_input.lower() != 'y':
                    print("사용자 요청으로 처리를 중단합니다.")
                    break
    
    print(f"\n모든 처리 완료. 처리된 이미지 수: {processed_images}")
    print(f"로그 파일: {log_file}")


os.chdir("../")
ROOT_DIR = os.getcwd()

# make dir preds3 at ROOT_DIR
os.makedirs(os.path.join(ROOT_DIR, "preds3"), exist_ok=True)

# 메인 실행 코드
if __name__ == "__main__":
    input_folder = os.path.join(ROOT_DIR, "preds2")
    output_folder = os.path.join(ROOT_DIR, "preds3")
    log_file = "processing_log.csv"
    
    process_folder(input_folder, output_folder, log_file, max_images=100)


처리 시작 (1/100): 0008fdb22ddce0ce_processed.jpg


NameError: name 'time' is not defined

In [7]:
import os
import cv2
import numpy as np
from PIL import Image
import csv
from datetime import datetime
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input

# Google Cloud Vision OCR 사용 (설정 필요)
from google.cloud import vision
import io

# 예상 키워드 및 문서 분류를 위한 키워드 사전
document_keywords = {
    "자동차 등록증": ["자동차등록증", "차대번호", "사용본거지", "자동차", "등록번호","등록증"],
    "진단서": ["진단서", "병명", "발병일", "향후치료소견", "의사", "면허번호"],
    "처방전": ["처방전", "조제시 참고사항", "용법", "조제량", "본인부담금", "보험자부담금"],
    "소견서": ["소견서", "진찰소견", "검사소견", "치료경과", "향후치료계획"],
    "임신 의료비 지급 신청서": ["임신", "출산", "의료비", "지급신청서", "요양기관", "진료과목"],
    "여권": ["REPUBLIC OF KOREA", "PASSPORT", "여권", "국적", "주민등록번호", "만기일"],
    "영수증": ["영수증", "금액", "공급가액", "부가세", "합계금액", "결제방법"],
    "주민등록증": ["주민등록증", "주민등록번호", "발행일", "주소", "시장"],
    "운전면허증": ["운전면허증", "면허번호", "조건", "적성검사", "운전면허", "경찰청"],
    "납부 확인서": ["납부확인서", "납부자", "납부일자", "납부금액", "수납기관"],
    "계좌번호": ["계좌번호", "예금주", "은행명", "계좌", "입금", "은행"],
    "입퇴원 확인서": ["입퇴원확인서", "입원일", "퇴원일", "진단명", "담당의","입원", "퇴원"],
    "외래 진료 확인서": ["외래진료확인서", "진료일시", "진료과목", "진료구분", "담당의사","진료확인서", "질병분류기호", "상병명", "진료기간", "치료내용","통원"],
    "자동차 번호판": ["[0-9]{2,3}[가-힣][0-9]{4}"],
    "자동차 대시보드": ["km/h", "rpm", "fuel", "온도", "ｏｉｌ", "battery"],
    "이력서": ["이력서", "학력사항", "경력사항", "자격증", "생년월일"],
    "진료비 영수증": ["진료비영수증", "환자", "진료과", "수납금액", "진료일"],
    "조제 영수증": ["조제영수증", "약제비", "조제료", "복약지도료", "의약품"]
}

# 형태소 분석기 초기화
okt = Okt()

# ResNet50 모델 로드 (가중치 제외)
image_model = ResNet50(weights=None, include_top=False, input_shape=(224, 224, 3))

def get_ocr_result_google(image_path):
    """Google Cloud Vision API를 사용하여 OCR 결과를 반환합니다."""
    client = vision.ImageAnnotatorClient()

    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations

    if texts:
        return texts[0].description, 1.0  # 신뢰도는 항상 1.0으로 가정
    else:
        return "", 0.0

def tokenize_text(text):
    """텍스트를 토큰화합니다."""
    return ' '.join(okt.morphs(text.lower()))

def calculate_similarity(text1, text2):
    """두 텍스트 간의 코사인 유사도를 계산합니다."""
    vectorizer = CountVectorizer().fit_transform([text1, text2])
    return cosine_similarity(vectorizer)[0][1]

def classify_document(text):
    """문서 유형을 추정합니다."""
    tokenized_text = tokenize_text(text)
    max_similarity = 0
    best_match = "Unknown"
    
    for doc_type, keywords in document_keywords.items():
        tokenized_keywords = ' '.join([tokenize_text(keyword) for keyword in keywords])
        similarity = calculate_similarity(tokenized_text, tokenized_keywords)
        if similarity > max_similarity:
            max_similarity = similarity
            best_match = doc_type
    
    return best_match, max_similarity

def get_image_features(image_path):
    """이미지의 특성을 추출합니다."""
    img = Image.open(image_path).resize((224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = image_model.predict(x)
    return features.flatten()

def process_image(image_path):
    """이미지를 처리하고 최적의 변형을 찾습니다."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"경고: 이미지를 읽을 수 없습니다 - {image_path}")
        return None, None, 0, 0, "", "Unknown", 0

    original_text, original_confidence = get_ocr_result_google(image_path)
    original_doc_type, original_similarity = classify_document(original_text)
    image_features = get_image_features(image_path)

    print(f"\n처리 중: {os.path.basename(image_path)}")
    print(f"  원본 Confidence: {original_confidence:.2f}")
    print(f"  추정 문서 유형: {original_doc_type} (유사도: {original_similarity:.2f})")
    print(f"  원본 텍스트: {original_text[:100]}..." if len(original_text) > 100 else f"  원본 텍스트: {original_text}")

    return image, "원본", original_confidence, original_similarity, original_text, original_doc_type, image_features

def process_folder(input_folder, output_folder, log_file):
    """폴더 내의 모든 이미지를 처리하고 결과를 로깅합니다."""
    os.makedirs(output_folder, exist_ok=True)
    
    with open(log_file, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['파일명', '최적 변형', 'OCR Confidence', '문서 유형 유사도', '추정 문서 유형', '텍스트', '처리 시간'])
        
        total_images = len([f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))])
        processed_images = 0
        
        for filename in os.listdir(input_folder):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                input_path = os.path.join(input_folder, filename)
                result = process_image(input_path)
                if result[0] is None:
                    print(f"오류: 이미지 처리 실패 - {filename}")
                    continue
                
                best_image, best_transform, confidence, similarity, text, doc_type, _ = result
                
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                
                output_path = os.path.join(output_folder, f"processed_{filename}")
                cv2.imwrite(output_path, best_image)
                
                csv_writer.writerow([filename, best_transform, confidence, similarity, doc_type, text, timestamp])
                
                processed_images += 1
                print(f"\n처리 완료: {processed_images}/{total_images} ({processed_images/total_images*100:.2f}%)")
    
    print(f"\n모든 처리 완료. 로그 파일: {log_file}")

os.chdir("../")
ROOT_DIR = os.getcwd()

# make dir preds3 at ROOT_DIR
os.makedirs(os.path.join(ROOT_DIR, "preds3"), exist_ok=True)

# 메인 실행 코드
if __name__ == "__main__":
    input_folder = os.path.join(ROOT_DIR, "preds2")
    output_folder = os.path.join(ROOT_DIR, "preds3")
    log_file = "processing_log.csv"
    
    process_folder(input_folder, output_folder, log_file)

ModuleNotFoundError: No module named 'google.cloud'

In [1]:
import os
import cv2
import numpy as np
import pytesseract
from PIL import Image
import csv
from datetime import datetime
import re

# 예상 키워드 및 문서 분류를 위한 키워드 사전 수정
document_keywords = {
    "자동차 등록증": ["자동차등록증", "차대번호", "사용본거지", "자동차", "등록번호","등록증"],
    "진단서": ["진단서", "병명", "발병일", "향후치료소견", "의사", "면허번호"],
    "처방전": ["처방전", "조제시 참고사항", "용법", "조제량", "본인부담금", "보험자부담금"],
    "소견서": ["소견서", "진찰소견", "검사소견", "치료경과", "향후치료계획"],
    "임신 의료비 지급 신청서": ["임신", "출산", "의료비", "지급신청서", "요양기관", "진료과목"],
    "여권": ["REPUBLIC OF KOREA", "PASSPORT", "여권", "국적", "주민등록번호", "만기일"],
    "영수증": ["영수증", "금액", "공급가액", "부가세", "합계금액", "결제방법"],
    "주민등록증": ["주민등록증", "주민등록번호", "발행일", "주소", "시장"],
    "운전면허증": ["운전면허증", "면허번호", "조건", "적성검사", "운전면허", "경찰청"],
    "납부 확인서": ["납부확인서", "납부자", "납부일자", "납부금액", "수납기관"],
    "계좌번호": ["계좌번호", "예금주", "은행명", "계좌", "입금", "은행"],
    "입퇴원 확인서": ["입퇴원확인서", "입원일", "퇴원일", "진단명", "담당의","입원", "퇴원"],
    "외래 진료 확인서": ["외래진료확인서", "진료일시", "진료과목", "진료구분", "담당의사","진료확인서", "질병분류기호", "상병명", "진료기간", "치료내용","통원"],
    "자동차 번호판": ["[0-9]{2,3}[가-힣][0-9]{4}"],
    "자동차 대시보드": ["km/h", "rpm", "fuel", "온도", "ｏｉｌ", "battery"],
    "이력서": ["이력서", "학력사항", "경력사항", "자격증", "생년월일"],
    "진료비 영수증": ["진료비영수증", "환자", "진료과", "수납금액", "진료일"],
    "조제 영수증": ["조제영수증", "약제비", "조제료", "복약지도료", "의약품"]
    
}

def get_ocr_result(image):
    """OCR 결과(텍스트와 신뢰도 점수)를 반환합니다."""
    ocr_result = pytesseract.image_to_data(image, lang='kor+eng', output_type=pytesseract.Output.DICT)
    text = ' '.join([word for word, conf in zip(ocr_result['text'], ocr_result['conf']) if int(conf) > -1])
    confidences = [int(conf) for conf in ocr_result['conf'] if conf != '-1']
    avg_confidence = np.mean(confidences) if confidences else 0
    return text, avg_confidence

def evaluate_text_quality(text):
    """텍스트의 품질을 평가합니다."""
    if not text.strip():
        return 0
    
    # 단어 수
    word_count = len(text.split())
    
    # 한글 문자 비율
    hangul_ratio = len(re.findall(r'[가-힣]', text)) / len(text) if len(text) > 0 else 0
    
    # 키워드 매칭 수 (모든 문서 유형의 키워드에 대해 검사)
    keyword_match = sum(1 for keywords in document_keywords.values() for keyword in keywords if keyword in text)
    
    # 품질 점수 계산
    quality_score = (word_count * 0.5) + (hangul_ratio * 30) + (keyword_match * 5)
    return quality_score

def classify_document(text):
    """문서 유형을 추정합니다."""
    max_score = 0
    best_match = "Unknown"
    
    for doc_type, keywords in document_keywords.items():
        score = sum(1 for keyword in keywords if keyword in text)
        if score > max_score:
            max_score = score
            best_match = doc_type
    
    return best_match

def process_image(image_path):
    """이미지를 처리하고 최적의 변형을 찾습니다."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"경고: 이미지를 읽을 수 없습니다 - {image_path}")
        return None, None, 0, 0, "", "Unknown"
    
    original_text, original_confidence = get_ocr_result(image)
    original_quality = evaluate_text_quality(original_text)
    print(f"\n처리 중: {os.path.basename(image_path)}")
    print(f"  원본 Confidence: {original_confidence:.2f}, 품질 점수: {original_quality:.2f}")
    print(f"  원본 텍스트: {original_text[:100]}..." if len(original_text) > 100 else f"  원본 텍스트: {original_text}")
    
    transformations = [
        ("원본", image),
        ("90도 회전", cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)),
        ("180도 회전", cv2.rotate(image, cv2.ROTATE_180)),
        ("270도 회전", cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)),
        ("좌우 대칭", cv2.flip(image, 1)),
        ("상하 대칭", cv2.flip(image, 0)),
    ]
    
    best_score = original_confidence * original_quality * 1.2  # 원본에 20% 가중치 부여
    best_image = image
    best_transform = "원본"
    best_text = original_text
    best_confidence = original_confidence
    best_quality = original_quality
    
    for transform_name, transformed_image in transformations:
        text, confidence = get_ocr_result(transformed_image)
        quality = evaluate_text_quality(text)
        score = confidence * quality
        print(f"  {transform_name:20s}: Confidence {confidence:.2f}, 품질 점수: {quality:.2f}")
        print(f"    텍스트: {text[:100]}..." if len(text) > 100 else f"    텍스트: {text}")
        if score > best_score:
            best_score = score
            best_image = transformed_image
            best_transform = transform_name
            best_text = text
            best_confidence = confidence
            best_quality = quality
    
    # 최종 처리 후 문서 유형 추정
    best_doc_type = classify_document(best_text)
    
    print(f"  최적 변형: {best_transform}")
    print(f"  최종 Confidence: {best_confidence:.2f}, 품질 점수: {best_quality:.2f}")
    print(f"  추정 문서 유형: {best_doc_type}")
    print(f"  최종 텍스트: {best_text[:100]}..." if len(best_text) > 100 else f"  최종 텍스트: {best_text}")
    return best_image, best_transform, best_confidence, best_quality, best_text, best_doc_type

def process_folder(input_folder, output_folder, log_file):
    """폴더 내의 모든 이미지를 처리하고 결과를 로깅합니다."""
    os.makedirs(output_folder, exist_ok=True)
    
    with open(log_file, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['파일명', '최적 변형', '최종 Confidence', '최종 품질 점수', '추정 문서 유형', '원본 Confidence', '원본 품질 점수', '개선도', '최종 텍스트', '처리 시간'])
        
        total_images = len([f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))])
        processed_images = 0
        
        for filename in os.listdir(input_folder):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                input_path = os.path.join(input_folder, filename)
                result = process_image(input_path)
                if result[0] is None:
                    print(f"오류: 이미지 처리 실패 - {filename}")
                    continue
                
                best_image, best_transform, best_confidence, best_quality, best_text, best_doc_type = result
                
                original_image = cv2.imread(input_path)
                if original_image is None:
                    print(f"오류: 원본 이미지를 읽을 수 없습니다 - {filename}")
                    continue
                
                original_text, original_confidence = get_ocr_result(original_image)
                original_quality = evaluate_text_quality(original_text)
                
                improvement = (best_confidence * best_quality) - (original_confidence * original_quality)
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                
                output_path = os.path.join(output_folder, f"processed_{filename}")
                cv2.imwrite(output_path, best_image)
                
                csv_writer.writerow([filename, best_transform, best_confidence, best_quality, best_doc_type, original_confidence, original_quality, improvement, best_text, timestamp])
                
                processed_images += 1
                print(f"\n처리 완료: {processed_images}/{total_images} ({processed_images/total_images*100:.2f}%)")
    
    print(f"\n모든 처리 완료. 로그 파일: {log_file}")


os.chdir("../")
ROOT_DIR = os.getcwd()

# make dir preds3 at ROOT_DIR
os.makedirs(os.path.join(ROOT_DIR, "preds3"), exist_ok=True)

# 메인 실행 코드
if __name__ == "__main__":
    input_folder = os.path.join(ROOT_DIR, "preds2")
    output_folder = os.path.join(ROOT_DIR, "preds3")
    log_file = "processing_log.csv"
    
    process_folder(input_folder, output_folder, log_file)


처리 중: 0008fdb22ddce0ce_processed.jpg
  원본 Confidence: 18.20, 품질 점수: 0.00
  원본 텍스트: 
  원본                  : Confidence 18.20, 품질 점수: 0.00
    텍스트: 
  90도 회전              : Confidence 18.20, 품질 점수: 0.00
    텍스트: 
  180도 회전             : Confidence 18.20, 품질 점수: 0.00
    텍스트: 
  270도 회전             : Confidence 18.20, 품질 점수: 0.00
    텍스트: 
  좌우 대칭               : Confidence 18.20, 품질 점수: 0.00
    텍스트: 
  상하 대칭               : Confidence 18.20, 품질 점수: 0.00
    텍스트: 
  최적 변형: 원본
  최종 Confidence: 18.20, 품질 점수: 0.00
  추정 문서 유형: Unknown
  최종 텍스트: 

처리 완료: 1/3140 (0.03%)

처리 중: 00091bffdffd83de_processed.jpg
  원본 Confidence: 18.20, 품질 점수: 0.00
  원본 텍스트: 
  원본                  : Confidence 18.20, 품질 점수: 0.00
    텍스트: 
  90도 회전              : Confidence -1.00, 품질 점수: 0.00
    텍스트: 
  180도 회전             : Confidence -1.00, 품질 점수: 0.00
    텍스트: 
  270도 회전             : Confidence 18.20, 품질 점수: 0.00
    텍스트: 
  좌우 대칭               : Confidence 18.20, 품질 점수: 0.00
    텍스트: 
  상하 대칭               : 

KeyboardInterrupt: 